In [1]:
import os
import sys
import h5py
import glob
import time
import numpy as np
import pandas as pd
import pickle as pickle
from pytz import timezone
import astropy.io.fits as pf
from astropy.time import Time
import matplotlib.pyplot as plt
from skyfield.api import load, wgs84, EarthSatellite

In [2]:
### FUNCTIONS NEEDED TO TRANSFORM PIXEL COORDS TO LOOK AT FOV

def world2pix(lst, ra, dec, jd=None, nx=4008, ny=2672, margin=50):
    xwcs, ywcs = world2wcs(wcspars, ra, dec, lst, jd) # Compute wcs-only pixel coordinates
    mask = np.isfinite(xwcs) & clip_rectangle(xwcs, ywcs, nx, ny, margin) # Remove coordinates not within the margins
    xwcs, ywcs = xwcs[mask], ywcs[mask]        
    xpix, ypix = wcs2pix(polpars, xwcs, ywcs) # Convert to actual pixel coordinates.
    return xpix, ypix, mask


def clip_rectangle(x, y, nx, ny, margin):
    maskx = (x > margin) & (x < (nx - margin))
    masky = (y > margin) & (y < (ny - margin))
    return maskx & masky


def world2wcs(wcspars, ra, dec, lst=None, jd=None):
    """ Convert world coordinates to WCS-only coordinates. """    
    if jd is not None:
        ra, dec = j2000_to_equinox(ra, dec, jd)
    w = create_wcs(wcspars, lst)
    xwcs, ywcs = w.wcs_world2pix(ra, dec, 0)
    return xwcs, ywcs


def create_wcs(wcspars, lst=None):
    """ Create and astropy WCS instance from a dictionary of parameters. """  
    from astropy import wcs
    
    if lst is not None:
        ra0, dec0 = wcspars['crval']
        ha0 = ra2ha(ra0, wcspars['lst'])
        ra0 = ha2ra(ha0, lst)
        crval = np.array([ra0, dec0])   
    else:
        crval = wcspars['crval']
        
    w = wcs.WCS(naxis=2)
    w.wcs.crpix = wcspars['crpix']
    w.wcs.cdelt = wcspars['cdelt']
    w.wcs.crval = crval
    w.wcs.ctype = ["RA---TAN", "DEC--TAN"]
    w.wcs.pc = wcspars['pc']
    return w


def ra2ha(ra, lst):
    """ Convert Right Ascension to Hour Angle. """
    ha = np.mod(lst*15. - ra, 360.)
    return ha

def ha2ra(ha, lst):
    """ Convert Hour Angle to Right Ascension. """    
    ra = np.mod(lst*15. - ha, 360.)
    return ra


def j2000_to_equinox(ra, dec, jd):
    from astropy.time import Time
    from astropy.coordinates import SkyCoord, FK5
    t = Time(jd, scale='utc', format='jd') 
    t.format = 'jyear_str'
    gc = SkyCoord(ra, dec, frame='fk5', unit='deg', equinox='J2000')    
    gc = gc.transform_to(FK5(equinox=t))    
    return gc.ra.value, gc.dec.value


def wcs2pix(polpars, xwcs, ywcs):
    """ Convert WCS-only coordinates to pixel coordinates. """     
    dx, dy = leg2d_eval(xwcs, ywcs, polpars['x_wcs2pix'], polpars['y_wcs2pix'], 
                        polpars['order'], polpars['nx'], polpars['ny'])    
    xpix, ypix = xwcs + dx, ywcs + dy
    return xpix, ypix


def leg2d_eval(x, y, a, b, order=6, nx=4008., ny=2672.):
    """ Evaluate 2-D polynomials. """
    mat, idx1, idx2 = leg2d_mat(x, y, order, nx, ny)
    xres = np.dot(mat, a[idx1, idx2])
    yres = np.dot(mat, b[idx1, idx2])
    return xres, yres  


def leg2d_mat(x, y, order=6, nx=4008., ny=2672.):
    """ Create a matrix for fitting 2-D polynomials of the given order. """ 
    from numpy.polynomial import legendre
    x = np.array(x)
    y = np.array(y)
    mat1 = legendre.legvander(2*(x/nx) - 1, order+1)
    mat2 = legendre.legvander(2*(y/ny) - 1, order+1)
        
    idx1, idx2 = np.indices((order+1, order+1))
    mask = ((idx1 + idx2) <= order)
    idx1, idx2 = idx1[mask], idx2[mask]        
        
    mat = mat1[:,idx1]*mat2[:,idx2]
    return mat, idx1, idx2

In [3]:
def reduce_tles(camid):

    # Load TLEs for all passages
    satfiles = '../test_data/3leComplete.txt'
    with open(satfiles) as f:
        all_tles = f.readlines()
        f.close()   

    # Split TLE list into individual lists for each TLE
    all_tles = [i.strip() for i in all_tles]
    tles = [all_tles[x:x+3] for x in range(0, len(all_tles), 3)]

    # Reduce TLEs to Starlink only
    starlink_tles = []
    for tle in tles:
        if "STARLINK" in tle[0]:
            starlink_tles.append(tle)

    # Obtain satellite passages
    passed_sats = pd.read_pickle(f'../test_data/passages/passed_satellites_20221023{camid}.p')

    # Find any Starlink TLEs in the passages
    idx = []
    flatlist = np.asarray(starlink_tles).flatten()
    for key in passed_sats.keys():
        line1 = passed_sats[key]['TLE line1'].strip()
        i = np.where(flatlist == line1)[0] 
        if i.size > 0:
            idx.append(i[0] - 1) #appending the name of the starlink sat

    # Now have indices for the flattened Starlink TLE list --> divide by 3 to get indices for the original list
    orig_idx = [int(x/3) for x in idx]
    passed_tles = [starlink_tles[i] for i in orig_idx]

    # Remove 0 labeling of first line of TLE because that's the proper format
    for tle in passed_tles:
        tle[0] = tle[0][2:]
    
    return passed_tles

In [4]:
# starlinks = reduce_tles('LSC')

In [5]:
def image_headers(camid):

    images = np.sort(glob.glob(f'../test_data/diff_images/{camid}/diff_*.fits.gz'))
    imginfo = {}
    for img in images:
        header = pf.getheader(img)
        lstseq = img[-19:-11]
        imginfo[lstseq] = {}
        imginfo[lstseq]['JD0'] = header['JD0']
        imginfo[lstseq]['JD1'] = header['JD1']
        imginfo[lstseq]['midLST'] = header['MIDLST']
        imginfo[lstseq]['midJD'] = header['MIDJD']
        imginfo[lstseq]['nx'] = header['XSIZE']
        imginfo[lstseq]['ny'] = header['YSIZE']

    return imginfo

In [6]:
def passage_check(midJD, tles, passages):
        
    idx_reduced = []
    sats = passages[midJD].keys()
    
    satnums = []
    for tle in tles:
        satnums.append(tle[1].split()[1])

    # Cross-reference
    for i, sat in enumerate(sats):
        if sat in satnums:
            idx = satnums.index(sat)
            idx_reduced.append(idx)
  
    return idx_reduced

In [7]:
def get_siteinfo():
    confdir = '../fotos-python3/bringfiles/siteinfo.dat'
    dtype = [('sitename', '|U20'), ('lat', 'float32'), ('lon', 'float32'), ('height', 'float32'), ('ID', '|U2')]
    siteinfo = np.genfromtxt(confdir, dtype=dtype)   
    mask = siteinfo['ID'] == 'LS'
    siteinfo = siteinfo[mask]
    return siteinfo

In [8]:
### Get timerange of all images ---- changing

def image_timerange(imgdata):
    ts = load.timescale()
    dates = []
    for lst in list(imgdata):
        dates.append(imgdata[lst]['midJD'])

    oldest = min(dates)
    newest = max(dates)

    t_old = Time(oldest, format='jd')
    t_new = Time(newest, format='jd')

    timerange = np.linspace(t_old, t_new, 150, endpoint=True) # every ~3mins

    return ts.from_astropy(timerange)

In [9]:
def check_illumination(imgdata, midJD, timerange, sat):

    # Check when satellite is illuminated 
    timerange = image_timerange(imgdata)
    sunlit = sat.at(timerange).is_sunlit(eph)

    # Obtain the indices of the first and last True element for each sequence of True elements 
    sunlit_idx = []
    start_idx = None
    for i, elem in enumerate(sunlit):
        if elem:
            # Checking if the start of a sequence of True elements
            if start_idx is None:
                start_idx = i
        else:
            # Checking if a sequence is just ended
            if start_idx is not None:
                sunlit_idx.append((start_idx, i-1)) # -1 because we're at the first False element after True sequence
                start_idx = None

    # If the last element is True, we need to append its index as well
    if start_idx is not None:
        sunlit_idx.append((start_idx, len(sunlit)-1))
    
    # Getting the times from the indices
    sunlit_times = []
    for idx in sunlit_idx:
        sunlit_times.append([timerange[idx[0]].to_astropy().value, timerange[idx[1]].to_astropy().value])
        #sunlit_times.append([timerange.tt[idx[0]], timerange.tt[idx[1]]])

    # Check if midJD is within each period and flag it if so
    illuminated = False
    for sunlitrng in sunlit_times:
        if midJD >= (sunlitrng[0] - 1/86400) and midJD <= (sunlitrng[1] + 1/86400): #adding a bumper of one second
            illuminated = True
            if illuminated:
                break

    return illuminated

## Need to keep the JD0 and JD1 for each image!

### I think we should re-write ProduceSkyPositions such that it:

- begins with an image
- extracts JD0, JD1, midJD
- reduces passages to those dates
- reduces the reduced passages to determined starlinks

In [89]:
#### NOTE

header = pf.getheader('../test_data/diff_images/LSC/diff_48506377LSC.fits.gz')
jd0 = header['JD0']
jd1 = header['JD1']
midJD = header[12]

# If we want all info such that: 
# Keys = midJDs: midJD --> [ [ jd0 -> [sats] ] , [ jd1 -> [sats] ] ] ... next midJD (each midJD is a spearate key)

sat1 = '48366U'
sat2 = '51788U'
v1 = {}
v1 = {midJD: {'JD0': {jd0: [sat1, sat2]}, 'JD1': {jd1: [sat1, sat2]}}}
print(v1)

# However, for reducing the passages in ProduceSkyPositions, it will be easier to just have:
# Keys = JD0s and JD1s: jd0 --> [sats] , jd1 --> [sats] , ... next JD0 and JD1 (each JD is a separate key)

v2 = {}
v2 = {'JD0': {jd0: [sat1, sat2]}, 'JD1': {jd1: [sat1, sat2]}}
print(v2)

{2459876.52443372: {'JD0': {2459876.524655353: ['48366U', '51788U']}, 'JD1': {2459876.524729224: ['48366U', '51788U']}}}
{'JD0': {2459876.524655353: ['48366U', '51788U']}, 'JD1': {2459876.524729224: ['48366U', '51788U']}}


## Should have LSTSEQ ????

Keys: LSTSEQs

Subkeys: JD0 and JD1 for that LSTSEQ (since corresponds to a single image)

- JD0 $\rightarrow$ [sat1, sat2, sat3, ...]
- JD1 $\rightarrow$ [sat1, sat2, sat3, ...]

... next LSTSEQ 


#### NOTE: I think I should also include midJD

Keys: LSTSEQs

Subkeys: JD0, JD1, midJD for that LSTSEQ (since corresponds to a single image)

- JD0 $\rightarrow$ [sat1, sat2, sat3, ...]
- JD1 $\rightarrow$ [sat1, sat2, sat3, ...]
- midJD $\rightarrow$ [sat1, sat2, sat3, ...]

... next LSTSEQ 

In [10]:
# Changing main function such that JD0s and JD1s are recorded WITH LSTSEQ

eph = load('de421.bsp')

def check_each_camid():
    imgdata = []
    times = []
    fasts = []
    psgs = []
    tles = []
    camids = ['LSC','LSE']
    pooldicts = [{}] * len(camids)
    
    print('Collecting image information\n')
    for camid in camids:
        d = image_headers(camid)
        imgdata.append(d)
        times.append(image_timerange(d))
        fasts.append(h5py.File(f"../fast_20221023{camid}.hdf5", "r"))
        psgs.append(pd.read_pickle(f"../test_data/passages/passages_20221023{camid}.p"))
        tles.append(reduce_tles(camid))
    
    # Define observer
    site = get_siteinfo()
    mascara = wgs84.latlon(latitude_degrees=site[0][1], longitude_degrees=site[0][2], elevation_m=site[0][3])
    
    print('Beginning Loop\n')
    for seq in fasts[0]['station']['lstseq'][:]: 
        lstseq = str(seq)

        for data, timerange, camid, fast, starlinks, passages, pool, in zip(
            imgdata, times, camids, fasts, tles, psgs, pooldicts):

            if lstseq not in data.keys():
                #print(f'{lstseq} not in directory')
                continue #this is only needed if not going through ALL images!

            # Set time of image
            midJD = data[lstseq]['midJD']
            ts = load.timescale()
            t = Time(midJD, format='jd')

            # Obtain passages
            idx_reduced = passage_check(midJD, starlinks, passages)

            #if idx_reduced is None: ## THIS IS WRONG!!!!!!
            if len(idx_reduced) == 0:
                print(f'{lstseq}: No starlinks passing MASCARA')
                continue

            # print(f'There are {len(idx_reduced)} Starlinks passing overhead {camid} for LSTSEQ={lstseq}')
            # Sun must be low enough below the horizon, otherwise data is not good enough
            observer = mascara + eph['earth']
            sun_pos = observer.at(ts.from_astropy(t)).observe(eph['sun'])
            sun_alt, sun_az, sun_dist = sun_pos.apparent().altaz()

            if sun_alt.degrees <= -18.:
                # Attaining the astrometric solution (depends on FAST file and LSTseq)
                astro = np.where((fast['astrometry/lstseq'][:] // 50) == (int(lstseq) // 50))[0][0]
                order = fast['astrometry/x_wcs2pix'][astro].shape[0]-1
                lst=fast['station/lst'][np.where(fast['station/lstseq'][:]==(fast['astrometry/lstseq'][astro]))[0][0]]
                nx = data[lstseq]['nx']
                ny = data[lstseq]['ny']

                wcspars = { 'crval' : fast['astrometry/crval'][astro].copy(),
                            'crpix' : fast['astrometry/crpix'][astro].copy(),
                            'cdelt' : [0.02148591731740587,0.02148591731740587],
                            'pc'    : fast['astrometry/pc'][astro].copy(),
                            'lst'   : lst }

                polpars = { 'x_wcs2pix' : fast['astrometry/x_wcs2pix'][astro].copy(),
                            'y_wcs2pix' : fast['astrometry/y_wcs2pix'][astro].copy(),
                            'x_pix2wcs' : fast['astrometry/x_pix2wcs'][astro].copy(),
                            'y_pix2wcs' : fast['astrometry/y_pix2wcs'][astro].copy(),
                            'nx'    : nx,
                            'ny'    : ny,
                            'order' : order }

                jd0 = data[lstseq]['JD0']
                jd1 = data[lstseq]['JD1']
                midLST = data[lstseq]['midLST']

                for idx in idx_reduced:
                    line1 = starlinks[idx][0]
                    line2 = starlinks[idx][1]
                    line3 = starlinks[idx][2] 
                    sat = EarthSatellite(line2, line3, line1, ts)

                    diff = sat - mascara
                    topocentric = diff.at(ts.from_astropy(t))
                    alt, az, dist = topocentric.altaz()

                    # Criteria check: if satellite is >20 degrees above the horizon
                    if alt.degrees >= 20:

                        # Criteria check: if satellite is illuminated
                        illuminated = check_illumination(data, midJD, timerange, sat)

                        if illuminated:
                            ra, dec, radec_dist = topocentric.radec() 
                            radeg = ra._degrees
                            dedeg = dec._degrees
                            ra, dec = j2000_to_equinox(radeg, dedeg, midJD)
                            w = create_wcs(wcspars, midLST)
                            xwcs, ywcs = w.wcs_world2pix(ra, dec, 0)
                            mask = np.isfinite(xwcs) & clip_rectangle(xwcs, ywcs, nx, ny, margin=50)

                            if mask:

                                if lstseq not in pool:
                                    pool[lstseq] = {}

                                if jd0 not in pool[lstseq]:
                                    pool[lstseq][jd0] = {}
                                    pool[lstseq][jd0] = [line2[2:8]]
                                else:
                                    if line2[2:8] not in pool[lstseq][jd0]:
                                        pool[lstseq][jd0].append(line2[2:8])

                                if jd1 not in pool[lstseq]:
                                    pool[lstseq][jd1] = {}
                                    pool[lstseq][jd1] = [line2[2:8]]
                                else:    
                                    if line2[2:8] not in pool[lstseq][jd1]:
                                        pool[lstseq][jd1].append(line2[2:8])

                                if midJD not in pool[lstseq]:
                                    pool[lstseq][midJD] = {}
                                    pool[lstseq][midJD] = [line2[2:8]]
                                else:    
                                    if line2[2:8] not in pool[lstseq][midJD]:
                                        pool[lstseq][midJD].append(line2[2:8])
                                            
                                


    for fast in fasts:
        fast.close()
        
    for (pool, camid) in zip(pooldicts, camids):
    
        if len(pool) > 0:
            pickle.dump(pool, open(f'../jd_test/pool_{camid}.p', 'wb'))  



In [16]:
check_each_camid()


Beginning Loop



In [98]:
passages = pd.read_pickle("../test_data/passages/passages_20221023LSC.p")
pool = pd.read_pickle('../jd_test/pool_LSC.p') # these are populated with JD0s, JD1s, and midJDs

In [100]:
# Reduce JDs in passages to just (jd0, jd1, midJD)
Y = {}
for JD, data in passages.items():
    if JD in [JD0, JD1, midJD]:
        Y[JD] = data

In [102]:
print([JD0, JD1, midJD])
print(list(pool['48506377']))
print(list(Y))

# Why Y begin wih midJD? Does it matter? (don't think so)

[2459876.520887864, 2459876.520961742, 2459876.520740109]
[2459876.520740109, 2459876.520887864, 2459876.520961742]


In [15]:
# Now reduce satnums in each JD of Y

Z = {}
for JD in Y:
    Z[JD] = {}
    for satnum in pool[lstseq][JD]:
        if satnum in Y[JD]:
            Z[JD][satnum] = Y[JD][satnum]

In [16]:
for jd in Z.keys():
    print(jd, list(Z[jd]))

2459876.52443372 ['48366U', '51788U']
2459876.524655353 ['48366U', '51788U']
2459876.524729224 ['48366U', '51788U']


### OKAY: 

We now have a working code that reduces the passages to JD0, JD1, midJD - and for only the satellites determined for each of these JDs.

So, we do this all iteratively for each LSTSEQ, i.e. for each image!

In [103]:
pool = pd.read_pickle('../selection_pool/pool_LSC.p')

### There should be 3 entries for each LSTSEQ.. (i.e. JD0, JD1, and midJD)

In [104]:
for i, lstseq in enumerate(pool.keys()):
    if len(pool[lstseq]) != 3:
        print(i, lstseq)
        print(list(pool[lstseq]))
        for jd in pool[lstseq]:
            print(list(pool[lstseq][jd]))

17 48506624
[2459876.542901684, 2459876.542975561]
['53603U', '47795U', '45733U']
['53603U', '47795U', '45733U']


In [105]:
pool['48506624']

{2459876.542901684: ['53603U', '47795U', '45733U'],
 2459876.542975561: ['53603U', '47795U', '45733U']}

In [106]:
for lstseq in pool.keys():
    if len(pool[lstseq]) != 3:
        for jd in pool[lstseq]:
            t = Time(jd, format='jd')
            print(t.iso) 
        print('\n')

2022-10-24 01:01:46.706
2022-10-24 01:01:53.088




### REASON FOUND! midJD = JD0. So, we will have to label the dictionary differently...

In [107]:
tles = reduce_tles('LSC')

In [116]:
pool['48506326']

{2459876.520887876: ['49424U', '47800U'],
 2459876.520961753: ['49424U', '47800U'],
 2459876.520740121: ['49424U', '47800U']}

In [118]:
header = pf.getheader(f'../selection_pool/diff_48506326LSC.fits.gz')
lstseq = '48506326'
midJD = header['midJD']
jd0 = header['JD0']
jd1 = header['JD1']

labs = ['JD0', 'JD1', 'mJD']

for i, jd in enumerate([jd0, jd1, midJD]):
    t = Time(jd, format='jd')
    print(labs[i], t.iso) 
    
len(tles)

JD0 2022-10-24 00:30:04.711
JD1 2022-10-24 00:30:11.095
mJD 2022-10-24 00:29:51.945


931

In [112]:
# # NEW check passages code

# idx_reduced = []
# sats_mid = passages[midJD].keys()
# sats_jd0 = passages[jd0].keys()
# sats_jd1 = passages[jd1].keys()

# satnums = []
# for tle in tles:
#     satnums.append(tle[1].split()[1])
    
# common_sats = set(sats_mid).intersection(sats_jd0, sats_jd1)

# # Cross-reference
# for sat in common_sats:
#     if sat in satnums:
#         idx = satnums.index(sat)
#         idx_reduced.append(idx)
        
# ## TEST

# test_mid = ['49447U', '44928U', '02481U', '02766U', '02971U']
# test_jd0 = ['02971U', '48264U', '40204U', '49447U', '44928U']
# test_jd1 = ['44928U', '40204U', '49447U', '40485U', '48264U']

# common_sats = set(test_mid).intersection(test_jd0, test_jd1)

# idx_reduced = []
# for sat in common_sats:
#     print(sat)
#     if sat in satnums:
#         idx = satnums.index(sat)
#         idx_reduced.append(idx)
        
# for idx in idx_reduced:
#     print(tles[idx])

Why is is that pool[lstseq] JDs don't match that of the image header info? Should be the same!!

In [119]:
from astropy.time import Time

for jd in Y:
    t = Time(jd, format='jd')
    print(t.iso)
    
print('\n')

for jd in pool[lstseq]:
    t = Time(jd, format='jd')
    print(t.iso)
    
# NOTE: ordering is 'wrong'

2022-10-24 00:29:51.945
2022-10-24 00:30:04.711
2022-10-24 00:30:11.095


2022-10-24 00:30:04.712
2022-10-24 00:30:11.095
2022-10-24 00:29:51.946


#### OKAY SO THE DIFFERENCE IS TEENY TINY.... numerical error? how to account for this???

It is possible that the slight difference is due to floating-point round-off errors, which is common in many programming languages including Python. One approach to handle this is to use a tolerance level when comparing the values

In [121]:
for lstseq in pool.keys():
    
    # For now, just selecting LSTSEQ we have
    header = pf.getheader(f'../selection_pool/diff_{lstseq}LSC.fits.gz')
    JD0 = header['JD0']
    JD1 = header['JD1']
    midJD = header['midJD']
    
    print(lstseq)
    print(JD0, JD1, midJD)
    
    Y = {}
    for date in [JD0, JD1, midJD]:
        for JD, data in passages.items():
            if date == JD:
                Y[JD] = data
                
    # nested for loop so as to make sure the 'correct' order is given
            
            
    # Now reduce satnums in each JD of Y
#     Z = {}
#     for JD in Y:
#         Z[JD] = {}
#         for satnum in pool[lstseq][JD]:
#             if satnum in Y[JD]:
#                 Z[JD][satnum] = Y[JD][satnum]
                
    break

48506326
2459876.520887864 2459876.520961742 2459876.520740109


In [122]:
for jd in Y:
    t = Time(jd, format='jd')
    print(jd, t.iso[11:])
    
print('\n')

for jd in pool[lstseq]:
    t = Time(jd, format='jd')
    print(jd, t.iso[11:])

2459876.520887864 00:30:04.711
2459876.520961742 00:30:11.095
2459876.520740109 00:29:51.945


2459876.520887876 00:30:04.712
2459876.520961753 00:30:11.095
2459876.520740121 00:29:51.946


In [124]:
# # Create a new dictionary with keys corrected for floating-point round-off errors

# import math
# T = 0.0000001

# Z = {}
# for yjd in Y:
    
#     Z[yjd] = {}
    
#     for pjd in pool[lstseq][yjd]:
        
#         if math.isclose(yjd, pjd, rel_tol=T):
#             print(pool[pjd])

In [55]:
pool[lstseq]

{2459876.520887876: ['49424U', '47800U'],
 2459876.520961753: ['49424U', '47800U'],
 2459876.520740121: ['49424U', '47800U']}

In [78]:
print(list(Y))
print(list(pool[lstseq]))

[2459876.520887864, 2459876.520961742, 2459876.520740109]
[2459876.520887876, 2459876.520961753, 2459876.520740121]


In [125]:
# Create a new dictionary with keys corrected for floating-point round-off errors

tolerance = 1e-6
Z = {}

for yJD in Y:
    for pJD in pool[lstseq]:
        
        if abs(yJD - pJD) <= tolerance:
            Z[pJD] = {}
            
            for sat in pool[lstseq][pJD]:
                
                Z[pJD][sat] = {}
                
                for k, v in Y[yJD].items():
                    if k == sat:
                        Z[pJD][sat][k] = v


In [126]:
Z

{2459876.520887876: {'49424U': {}, '47800U': {}},
 2459876.520961753: {'49424U': {}, '47800U': {}},
 2459876.520740121: {'49424U': {}, '47800U': {}}}

### Description of problem:

#### First issue: midJD < JD0

In [129]:
lstseq = '48506326'

header = pf.getheader(f'../selection_pool/diff_{lstseq}LSC.fits.gz')
midJD = header['midJD']
jd0 = header['JD0']
jd1 = header['JD1']

labs = ['JD0', 'JD1', 'mJD']
for i, jd in enumerate([jd0, jd1, midJD]):
    t = Time(jd, format='jd')
    print(labs[i], t.iso) 

JD0 2022-10-24 00:30:04.711
JD1 2022-10-24 00:30:11.095
mJD 2022-10-24 00:29:51.945


In [134]:
# Reduce Passages file (keys = JDs) to the given JDs

Y = {}
for date in [JD0, JD1, midJD]:
    for JD, data in passages.items():
        if date == JD:
            Y[JD] = data
            
print(f'Passages reduced from {len(passages)} to {len(Y)}')

Passages reduced from 4492 to 3


In [140]:
# Let's look at the selection pool for the LSTSEQ we're testing:
pool[lstseq]

{2459876.520887876: ['49424U', '47800U'],
 2459876.520961753: ['49424U', '47800U'],
 2459876.520740121: ['49424U', '47800U']}

We now want to reduce the satellites in Y to those in our selection pool (while keeping all the relevant info from Y such as positions etc). We do this by matching the JDs in both dictionaries - second issue!

#### Second issue: floating-point round-off errors

In [141]:
for jd in Y:
    t = Time(jd, format='jd')
    print(jd, t.iso[11:])
    
print('\n')

for jd in pool[lstseq]:
    t = Time(jd, format='jd')
    print(jd, t.iso[11:])

2459876.520887864 00:30:04.711
2459876.520961742 00:30:11.095
2459876.520740109 00:29:51.945


2459876.520887876 00:30:04.712
2459876.520961753 00:30:11.095
2459876.520740121 00:29:51.946


In [142]:
# We account for this by adding a tolerance:

tolerance = 1e-6
Z = {}

for yJD in Y:
    for pJD in pool[lstseq]:
        
        if abs(yJD - pJD) <= tolerance:
            Z[pJD] = {}
            
            for sat in pool[lstseq][pJD]:
                
                Z[pJD][sat] = {}
                
                for k, v in Y[yJD].items():
                    if k == sat:
                        Z[pJD][sat][k] = v


In [143]:
Z

{2459876.520887876: {'49424U': {}, '47800U': {}},
 2459876.520961753: {'49424U': {}, '47800U': {}},
 2459876.520740121: {'49424U': {}, '47800U': {}}}

#### Z is empty?

In [146]:
sats = ['49424U', '47800U']

for sat in sats:
    if sat not in Y[jd0]:
        print('Satellite not found')
        
for sat in sats:
    if sat not in Y[jd1]:
        print('Satellite not found')
        
for sat in sats:
    if sat not in Y[midJD]:
        print('Satellite not found')

Satellite not found
Satellite not found
Satellite not found
Satellite not found
Satellite not found
Satellite not found


#### Third issue: the satellites in our selection pool are not found in the passage file for their respective JD. ???

In [150]:
passed = pd.read_pickle("../test_data/passages/passed_satellites_20221023LSC.p")

In [155]:
for sat in sats:
    if sat not in passages.keys():
        print('Nope!')

Nope!
Nope!


Interesting, the satellites in the selection pool are not in the passed satellites files either..

In [159]:
passed_sats = passed.keys()

In [165]:
match = []

for tle in tles:
    if tle[1][2:8] in passed_sats:
        match.append(tle[1][2:8])

In [167]:
print(len(tles))
print(len(match))

931
931


In [169]:
for sat in sats:
    if sat in match:
        print('Yes')

#### STRANGE: the satellites in the selection pool are not within the Starlink TLEs! This is a BIG error

In [174]:
imgdata = []
times = []
psgs = []
starlinks = []
camids = ['LSC','LSE']

for camid in camids:
    d = image_headers(camid)
    imgdata.append(d)
    times.append(image_timerange(d))
    psgs.append(pd.read_pickle(f"../test_data/passages/passages_20221023{camid}.p"))
    starlinks.append(reduce_tles(camid))

In [182]:
satfiles = '../test_data/3leComplete.txt'
with open(satfiles) as f:
    all_tles = f.readlines()
    f.close()   

# Split TLE list into individual lists for each TLE
all_tles = [i.strip() for i in all_tles]
alltles = [all_tles[x:x+3] for x in range(0, len(all_tles), 3)]

In [193]:
satnums = []
for tle in alltles:
    satnums.append(tle[1][2:8])
    
for sat in sats:
    if sat in satnums:
        print('True')

True
True


In [196]:
# Reduce TLEs to Starlink only
starlink_tles = []
for tle in alltles:
    if "STARLINK" in tle[0]:
        starlink_tles.append(tle)
        
len(starlink_tles)

3179

In [270]:
satnums = []
for tle in starlink_tles:
    satnums.append(tle[1][2:8])

test = []
for i, satnum in enumerate(satnums):
    if satnum in list(passed):
        test.append(i-1)

In [254]:
%%timeit
idx = []
flatlist = np.asarray(starlink_tles).flatten()
line1_list = [passed[key]['TLE line1'].strip() for key in passed.keys()]
idx = [np.where(flatlist == line)[0][0] - 1 for line in line1_list if np.where(flatlist == line)[0].size > 0]

292 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [271]:
%%timeit
idx = []
flatlist = np.asarray(starlink_tles).flatten()
for key in passed.keys():
    line1 = passed[key]['TLE line1'].strip()
    i = np.where(flatlist == line1)[0] 
    if i.size > 0:
        idx.append(i[0] - 1) 

260 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [272]:
if test == idx:
    print('yes')

yes


In [259]:
# Now have indices for the flattened Starlink TLE list --> divide by 3 to get indices for the original list
orig_idx = [int(x/3) for x in idx]
passed_tles = [starlink_tles[i] for i in orig_idx]

# Remove 0 labeling of first line of TLE because that's the proper format
for tle in passed_tles:
    tle[0] = tle[0][2:]

In [278]:
for i, tle in enumerate(alltles):
    if sats[0] == tle[1][2:8]:
        print(sats[0], i)
    if sats[1] == tle[1][2:8]:
        print(sats[1], i)

47800U 19292
49424U 20699


In [281]:
for i, tle in enumerate(starlink_tles):
    if sats[0] == tle[1][2:8]:
        print(sats[0], i)
    if sats[1] == tle[1][2:8]:
        print(sats[1], i)

47800U 1019
49424U 1587


In [283]:
for i, sat in enumerate(list(passed)):
    if sats[0] == sat:
        print(sats[0], i)
    if sats[1] == sat:
        print(sats[1], i)

In [294]:
satnums = []
for tle in tles:
    satnums.append(tle[1].split()[1])

idx_reduced = []
for i, sat in enumerate(passages[midJD].keys()):
    if sat in satnums:
        print(sat)
        idx = satnums.index(sat)
        idx_reduced.append(idx)

48367U
51790U


In [297]:
idx_reduced = []
sats_mid = passages[midJD].keys()
sats_jd0 = passages[jd0].keys()
sats_jd1 = passages[jd1].keys()

satnums = []
for tle in tles:
    satnums.append(tle[1].split()[1])

common_sats = set(sats_mid).intersection(sats_jd0, sats_jd1)

# Cross-reference
for sat in common_sats:
    if sat in satnums:
        print(sat)
        idx = satnums.index(sat)
        idx_reduced.append(idx)
        
idx_reduced

48367U
51790U


[10, 12]

In [293]:
tles[10]

['STARLINK-2643',
 '1 48367U 21038Q   22294.99038760 -.00000807  00000-0 -35314-4 0  9990',
 '2 48367  53.0539 134.4060 0001353 125.3004 234.8111 15.06396799 81060']

In [299]:
pool[lstseq]

{2459876.520887876: ['49424U', '47800U'],
 2459876.520961753: ['49424U', '47800U'],
 2459876.520740121: ['49424U', '47800U']}